# Packages

In [3]:
%pip install statsbombpy


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import json
import time
from pathlib import Path
from datetime import datetime
import ast

import pandas as pd
import geopandas as gpd
import numpy as np
from statsbombpy import sb

# Loading data

In [5]:
# loading the liga games from the 2015/16 season
competitions = sb.competitions()[(sb.competitions()["season_name"] == "2015/2016") & (sb.competitions()["competition_name"] != "Champions League")]["competition_id"]
season_id = 27 # for 2015/16 season
competitions

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


1      9
43    11
60     7
64     2
66    12
Name: competition_id, dtype: int64

In [6]:
match_ids = {}
for competition_id in competitions:
    match_ids[competition_id] = list(sb.matches(competition_id=competition_id, season_id=season_id)["match_id"])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [7]:
match_ids

{9: [3890561,
  3890505,
  3890511,
  3890515,
  3890411,
  3890397,
  3890401,
  3890396,
  3890384,
  3890385,
  3890282,
  3890287,
  3890269,
  3890268,
  3890402,
  3890564,
  3890563,
  3890562,
  3890560,
  3890559,
  3890558,
  3890557,
  3890556,
  3890555,
  3890554,
  3890553,
  3890552,
  3890551,
  3890550,
  3890549,
  3890548,
  3890547,
  3890546,
  3890545,
  3890544,
  3890543,
  3890542,
  3890541,
  3890540,
  3890539,
  3890538,
  3890537,
  3890536,
  3890535,
  3890534,
  3890533,
  3890532,
  3890531,
  3890530,
  3890529,
  3890528,
  3890527,
  3890526,
  3890525,
  3890524,
  3890523,
  3890522,
  3890521,
  3890520,
  3890519,
  3890518,
  3890517,
  3890516,
  3890514,
  3890513,
  3890512,
  3890510,
  3890509,
  3890508,
  3890507,
  3890506,
  3890504,
  3890503,
  3890502,
  3890501,
  3890500,
  3890499,
  3890498,
  3890497,
  3890496,
  3890495,
  3890494,
  3890493,
  3890492,
  3890491,
  3890490,
  3890489,
  3890488,
  3890487,
  3890486,
  38904

In [31]:
sb.events(match_id=3890557).loc[sb.events(match_id=3890557)["shot_one_on_one"].isna()==False, "shot_one_on_one"]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


3011    True
Name: shot_one_on_one, dtype: object

In [ ]:
grouped_events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2015/2016",
    split=True
)
grouped_events["shots"]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Pytho

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,shot_aerial_won,shot_one_on_one,shot_deflected,out,shot_open_goal,shot_redirect,shot_saved_off_target,shot_saved_to_post,off_camera,shot_follows_dribble
0,ba46e9d6-e828-4599-952c-39c1f7d22659,263,1,00:04:21.052,4,21,Shot,7,Hoffenheim,Regular Play,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85d67225-30fb-47c8-b478-cf568941a164,353,1,00:06:27.395,6,27,Shot,11,Schalke 04,From Throw In,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,adac17d3-5e67-4e8c-b482-4bae2f36e06e,410,1,00:08:16.762,8,16,Shot,15,Hoffenheim,From Throw In,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abffd193-62bc-4c8d-8636-1e3f0f0ebbe5,624,1,00:13:55.721,13,55,Shot,27,Schalke 04,From Counter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9cea903-f92a-40e1-a393-1a849d83f157,749,1,00:17:16.953,17,16,Shot,33,Schalke 04,Regular Play,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,771fb93c-9f8a-4aae-978c-39bee95b7cf1,2993,2,00:27:14.850,72,14,Shot,138,Bayern Munich,From Counter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7827,21d1b0e1-4131-49a3-b0e0-2f38fafd4147,3185,2,00:32:17.918,77,17,Shot,145,Bayern Munich,From Throw In,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7828,6a80fdad-e1da-42d5-adfe-855664f36622,3435,2,00:38:16.394,83,16,Shot,156,Bayern Munich,From Free Kick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7829,a75af5cb-a837-44b3-8d15-9251b11d3d4a,3545,2,00:41:15.431,86,15,Shot,163,Bayern Munich,From Goal Kick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
